# Compute mean model

### Load pyaerocom and set base directory

In [2]:
import pyaerocom as pya

pya.const.BASEDIR = '/home/notebook/shared-ns1000k/inputs/pyaerocom-testdata/'

Initiating directories for pyaerocom test dataset


### Specify year, variable, frequency and models to be merged

In [3]:
YEAR = 2010
VAR_NAME = 'od550aer'
TS_TYPE = 'monthly'

In [8]:
MODEL_IDS = ['CAM6-Oslo_NHIST_f19_tn14_20190710_2010', 'ECHAM6.3-HAM2.3-met2010_AP3-CTRL', 'CAM5-ATRAS_AP3-CTRL']

MODEL_IDS = ['CAM6-Oslo_NHIST_f19_tn14_20190710_2010', 'ECHAM6.3-HAM2.3-met2010_AP3-CTRL']#, 'CAM5-ATRAS_AP3-CTRL']

### (Lazy) load model fields

In [10]:
data = []
for model in MODEL_IDS:
    data.append(pya.io.ReadGridded(model).read_var(VAR_NAME, start=YEAR, ts_type='monthly'))

In [11]:
for model in data:
    print(model.data_id, model.shape) 

CAM6-Oslo_NHIST_f19_tn14_20190710_2010 (11, 96, 144)
ECHAM6.3-HAM2.3-met2010_AP3-CTRL (12, 96, 192)


In [14]:
import xarray as xr

arr = xr.load_dataset(data[0].from_files[0])
arr

<xarray.Dataset>
Dimensions:    (lat: 96, lon: 144, nbnd: 2, time: 12)
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) object 2010-02-01 00:00:00 ... 2011-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables:
    od550aer   (time, lat, lon) float32 0.003288469 0.0032884297 ... 0.030857734
    gw         (lat) float64 0.0001367 0.001093 0.002185 ... 0.001093 0.0001367
    time_bnds  (time, nbnd) object 2010-01-01 00:00:00 ... 2011-01-01 00:00:00
Attributes:
    Conventions:                CF-1.0
    source:                     CAM
    case:                       NHIST_f19_tn14_20190710
    logname:                    olivie
    host:                       
    initial_file:               /cluster/shared/noresm/inputdata/atm/cam/inic...
    topography_file:            /cluster/shared/noresm/inputdata/atm/cam/topo...
    model_doi_url:

In [41]:
model.time_stamps()

array(['2010-02-01T00:00:00.000000', '2010-03-01T00:00:00.000000',
       '2010-04-01T00:00:00.000000', '2010-05-01T00:00:00.000000',
       '2010-06-01T00:00:00.000000', '2010-07-01T00:00:00.000000',
       '2010-08-01T00:00:00.000000', '2010-09-01T00:00:00.000000',
       '2010-10-01T00:00:00.000000', '2010-11-01T00:00:00.000000',
       '2010-12-01T00:00:00.000000'], dtype='datetime64[us]')

In [26]:
num_gp = []
for mod in data:
    num_gp.append(len(mod.longitude.points) * len(mod.latitude.points))
num_gp

[18432, 13824]

In [27]:
data[0]

pyaerocom.GriddedData
Grid data: <iris 'Cube' of Ambient Aerosol Optical Thickness at 550 nm / (1) (time: 12; latitude: 96; longitude: 192)>

In [28]:
data[1]

pyaerocom.GriddedData
Grid data: <iris 'Cube' of ambient aerosol optical thickness at 550 nm / (1) (time: 12; latitude: 96; longitude: 144)>

In [29]:
data_sorted = [x for _,x in sorted(zip(num_gp,data))]

In [30]:
data_sorted[0]

pyaerocom.GriddedData
Grid data: <iris 'Cube' of ambient aerosol optical thickness at 550 nm / (1) (time: 12; latitude: 96; longitude: 144)>

In [31]:
data_sorted[1]

pyaerocom.GriddedData
Grid data: <iris 'Cube' of Ambient Aerosol Optical Thickness at 550 nm / (1) (time: 12; latitude: 96; longitude: 192)>

In [32]:
base_model = data_sorted.pop(0)

In [33]:
data_sorted

[pyaerocom.GriddedData
 Grid data: <iris 'Cube' of Ambient Aerosol Optical Thickness at 550 nm / (1) (time: 12; latitude: 96; longitude: 192)>]

In [34]:
for md in data_sorted:
    base_model = md.regrid(base_model)